# Differential gene expression analysis


## Background

The project focuses on differential gene expression testing on complex experimental designs which involve one or more conditions such as diseases, genetic knockouts or drugs. Specifically, we aims to identify differentially expressed genes in response to BMP4 (Bone Morphogenetic Protein 4) perturbation using the gehring_2019 single-cell RNA-seq dataset.

For detailed tutorial, please refer to https://www.sc-best-practices.org/conditions/differential_gene_expression.html. In this project, we will follow the tutorial and perform differential gene expression analysis for another dataset gehring_2019() (https://www.nature.com/articles/s41587-019-0372-z). While you are reading the tutorial, you can start the environment setup in the Google colab. This might take a while.

## Environment setup

In [ ]:
!pip install anndata2ri
!pip install pertpy
!pip install sc_toolbox
!pip install rpy2
!pip install scanpy
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 77.7 MB/s eta 0:00:00


In [1]:
import warnings

warnings.filterwarnings("ignore")

import logging
import random

import anndata2ri
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pertpy
import rpy2.rinterface_lib.callbacks
import sc_toolbox
import scanpy as sc
import seaborn as sns
from rpy2.robjects import pandas2ri

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("edgeR")
BiocManager::install("S4Vectors")
BiocManager::install("SingleCellExperiment")
library(edgeR)
library("S4Vectors")
library("SingleCellExperiment")

## Preparing the dataset

We will use the gehring dataset, which is a 96-plex perturbation experiment on live mouse neural stem cells, consisting of a pair of drug-triples with 4 drugs in total at 3 or 4 different concentractions.

First, we load the full dataset.

In [4]:
mdata = pertpy.data.gehring_2019()
mdata

Output()

AnnData object with n_obs × n_vars = 20382 × 13256
    obs: 'batch', 'disease', 'cancer', 'tissue_type', 'celltype', 'perturbation', 'perturbation_2', 'perturbation_3', 'perturbation_4', 'dose_unit', 'dose_unit_2', 'dose_unit_3', 'dose_unit_4', 'organism', 'perturbation_type', 'dose_value', 'dose_value_2', 'dose_value_3', 'dose_value_4', 'nperts', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [5]:
mdata.obs[:5]

,batch,disease,cancer,tissue_type,celltype,perturbation,perturbation_2,perturbation_3,perturbation_4,dose_unit,...,perturbation_type,dose_value,dose_value_2,dose_value_3,dose_value_4,nperts,ncounts,ngenes,percent_mito,percent_ribo
AAACCTGCACACATGT,0,healthy,False,primary,neural stem cells,BMP4,EGF and bFGF,1:5 Scriptaid:decitabine,retinoic acid,ng/mL,...,drug,8,200.0,0.0,0,2,2705.258789,1761,0.061296,1.013085
AAACCTGCACGTCAGC,0,healthy,False,primary,neural stem cells,BMP4,EGF and bFGF,1:5 Scriptaid:decitabine,retinoic acid,ng/mL,...,drug,0,40.0,0.0,10,2,2583.522705,2062,0.056831,1.000418
AAACCTGCATTGGTAC,0,healthy,False,primary,neural stem cells,BMP4,EGF and bFGF,1:5 Scriptaid:decitabine,retinoic acid,ng/mL,...,drug,8,200.0,0.0,2,3,2731.804199,2080,0.086957,0.833775
AAACCTGGTCGCATAT,0,healthy,False,primary,neural stem cells,BMP4,EGF and bFGF,1:5 Scriptaid:decitabine,retinoic acid,ng/mL,...,drug,40,8.0,0.0,2,3,2661.003174,1925,0.106220,0.911778
AAACCTGGTGCAGGTA,0,healthy,False,primary,neural stem cells,BMP4,EGF and bFGF,1:5 Scriptaid:decitabine,retinoic acid,ng/mL,...,drug,0,40.0,0.0,10,2,3084.830566,2646,0.112301,1.025654


We will need `batch`  and `dose_value` columns of the `.obs`.

In [6]:
mdata.obs = mdata.obs[["batch", "dose_value"]]

In [7]:
mdata

AnnData object with n_obs × n_vars = 20382 × 13256
    obs: 'batch', 'dose_value'
    var: 'ensembl_id', 'ncounts', 'ncells'

We will need to work with raw counts so we check that `.X` indeed contains raw counts and put them into the `counts` layer of our AnnData object.

In [9]:
np.max(mdata.X)

np.float32(7.4750285)

In [10]:
mdata.layers["counts"] = mdata.X.copy()

Check out how many batches we have.

In [11]:
mdata.obs["batch"].value_counts()

,count
batch,
0,9249
1,8755
3,1226
2,1152


In [12]:
mdata.obs["dose_value"] = mdata.obs["dose_value"].astype("category")

In [14]:
print(len(mdata[mdata.obs["batch"] == "0"].obs["dose_value"].cat.categories))
print(len(mdata[mdata.obs["batch"] == "1"].obs["dose_value"].cat.categories))
print(len(mdata[mdata.obs["batch"] == "2"].obs["dose_value"].cat.categories))
print(len(mdata[mdata.obs["batch"] == "3"].obs["dose_value"].cat.categories))

3
3
3
3


We filter cells which have less than 200 genes and genes which were found in less than 3 cells for a rudimentary quality control.

In [15]:
sc.pp.filter_cells(mdata, min_genes=200)
sc.pp.filter_genes(mdata, min_cells=3)
mdata

AnnData object with n_obs × n_vars = 20382 × 13255
    obs: 'batch', 'dose_value', 'n_genes'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells'
    layers: 'counts'

## Clustering

First of all, we perform clustering on the dataset to see if we get the same figure as in the article. You may want to refer to this tutorial, https://www.sc-best-practices.org/cellular_structure/clustering.html.


## Pseudobulk

Since we need to create pseudobulks for each batch-dose combination, we first need to create such a column by concatenating `batch` and `dose_value` (for perturbation BMP4).

We need to set categorical metadata to be indeed categorical to create pseudobulks.

### One group

First, we show how to prepare the data, construct the design matrix and perform the DE testing for perturbation BMP4.
